# importar librerías y settings

In [ ]:
# importar librerías
import re, os, pandas as pd, numpy as np

# Importar UDFs
import anexo_5_tables_udfs as tf

# comprobar WD
print(os.getcwd())

# ampliar visualizaciones de pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

***
### tabla_5

In [ ]:
# importar
df_path = "../Anexo_4_full_text_screening/Anexo_4_carpeta_1_resultados/tabla_4.6_stats.csv"
df = pd.read_csv(df_path, sep=",")

# castear las últimas columnas a tipo int
for col in df.columns[1:]:
    df[col] = df[col].astype('Int64')

# check
df

### preprocesar datos

In [ ]:
# imprimimos los valores de "index" para modificarlos y usarlos directamente en el flowchart
print(list(df["index"]))

# almacenamos valores modificados en una lista
texts = ['#1: Términos generales Enfermedades Renales Hereditarias',
         '#2: Términos ADPKD',
         '#3: Genes asociados', 
         '#4: Técnicas ómicas y análisis bioinformático',
         '#5: Biomarcadores y aplicaciones traslacionales',
         'Registros identificados mediante\nbúsqueda en bases de datos', 
         '• Estudios de cáncer', 
         '• Anteriores a 2015',
         '• No publicados en inglés',
         '• Duplicados o retractados', 
         '• Publicaciones secundarias',
         'Excluidos en revisión de títulos y abstracts',
         '• No centrados en ADPKD',
         '• Sin análisis específico de ADPKD',
         '• Sin biomarcadores relevantes',
         '• Sin pacientes humanos',
         'Excluidos en revisión de textos completos',
         '• Publicaciones secundarias',
         '• Texto completo inaccesible',
         '• Sin análisis específico de ADPKD',
         '• Revisiones sin datos originales']

# y se la asignamos a la columna de turno
df["index"]=texts

# añadir tipos de estudios incluidos para flowchart
## importamos el minado de datos de los estudios
st_min_path = "../Anexo_4_full_text_screening/Anexo_4_carpeta_1_resultados/tabla_4.4_studies_extraction.csv"
st_min = pd.read_csv(st_min_path, sep=";")

# hacemos un df con el value_counts() de los tipos de estudio
sd = st_min["study_design"].value_counts()
sd = sd.reset_index()
sd.columns = ["index", "Total"]

# Añadir como fila 0 el total
temp_df = {"index": ["Incluidos: full text review"], "Total": [sum(sd["Total"])]}

# pasar a df (y trasponer)
temp_df = pd.DataFrame(temp_df)
temp_df["Total"] = temp_df["Total"].astype("Int64")

# concateno los dos
temp_df = pd.concat([temp_df, sd])

# renombro filas de sd con 1ª col
temp_df.columns = ["index", "Total"]

# concatenar a stats
df = pd.concat([df, temp_df], axis=0).reset_index().drop(columns=["level_0"])

In [ ]:
# check
df

In [ ]:
# exportar stats
stats_path="./Anexo_5_carpeta_1_resultados/tabla_5_stats.csv"
df.to_csv(stats_path, sep=";", index=False)

***
# Tabla 1: construcción de la query de la búsqueda

In [ ]:
# definimos ruta donde están los constructos
construct_dir = '../Anexo_1_construir_queries/Anexo_1_carpeta_1_constructos'

# creamos un dict. vacío para almacenar resultados
construct_dict = {}

for file in os.listdir(construct_dir):
    filename = os.fsdecode(file)
    if filename.endswith(".txt"):
        
        # extraemos nombre del constructo (entre el penúltimo "_" y ".txt")
        clave = filename.split('_')[-1].replace('.txt', '')

        # leemos contenido del archivo .txt con with open y lo guardamos en la variable valores
        with open(os.path.join(construct_dir, filename), "r", encoding="utf-8") as f:
            valores = f.read()
            
            # quitar encabezados de los constructos, separadores "OR" + texto circundante y \W innecesarios
            to_erase = [r'^#.*(?:\r?\n)+', r'\"\[\w+\]\s\w+\s\"', r"([\(\)\"])|(\[\w+\])?"]
            for i,regex in enumerate(to_erase):
                if i == 1:
                    valores = re.sub(regex, ', ', valores)
                else:
                    valores = re.sub(regex, '', valores)
            
            # convertir valores en lista
            valores = valores.split(", ")
        
        # asignamos clave y valores a construct_dict
        construct_dict[clave] = valores

# encontrar la longitud máxima de cada lista (para transformar dict en df todos los valores deben tener el mismo largo)
max_len = max(len(valores) for valores in construct_dict.values())

# rellenar listas cortas con ''
for clave in construct_dict:
    construct_dict[clave] += [''] * (max_len - len(construct_dict[clave]))

# creamos dataframe (que convertiremos en tabla1)
tabla1 = pd.DataFrame(construct_dict)

# renombrar columnas
tabla1 = tabla1.rename(columns={"biomarcadores": "Constructo Biomarcadores", 
                                "diseases": "Constructo Enfermedades Renales Hereditarias",
                                "computacional": "Constructo Ómicas y Biología Computacional",
                                "adpkd": "Constructo ADPKD",
                                "genes": "Constructo Genes Asociados"})

# poner primera letra de cada palabra en mayus (las últimas 2 cols no lo necesitan)
for col in tabla1.columns[:3]:
    tabla1[col] = tabla1[col].str.title()

# re-ordenar columnas
cols = list(tabla1.columns)
cols = ['Constructo Enfermedades Renales Hereditarias', 'Constructo ADPKD', 'Constructo Genes Asociados', 
        'Constructo Ómicas y Biología Computacional', 'Constructo Biomarcadores']
tabla1 = tabla1[cols]

# limpiar tabla (sustituir y limpiar valores)
## col 0
tabla1.iloc[0:5,0] = tabla1.iloc[2:7,0]
tabla1.iloc[6:8,0] = ""

## col 3
tabla1.iloc[0:17,3] = tabla1.iloc[2:,3]
tabla1.iloc[17:,3] = ""

## col 4
tabla1.iloc[1:6,4] = tabla1.iloc[2:7,4]
tabla1.iloc[6,4] = ""

## eliminar filas vacías
tabla1 = tabla1.drop([17, 18])

# check
tabla1

In [ ]:
# exportar
tabla1_path = "./Anexo_5_carpeta_1_resultados/tabla_1_constructs.csv"
tabla1.to_csv(tabla1_path, index=False, sep=";")

***
# Tabla 2: resultados de la búsqueda

In [ ]:
# importar search_stats
search_stats_path = "./Anexo_5_carpeta_1_resultados/tabla_5_stats.csv"
search_stats = pd.read_csv(search_stats_path, sep=";", index_col=False)

# quedarnos solo con las primeras 6 filas y eliminar columna "Excluidos" (que va en el flowchart)
tabla2 = search_stats.iloc[:10,:].drop(columns=["Excluidos"])

# castear columnas numéricas a tipo entero
for col in tabla2.columns[-3:]:
    tabla2[col] = tabla2[col].astype("Int64")

# renombrar col. "index" a "Constructo"
tabla2 = tabla2.rename(columns={"index": "Constructo"})

# reasignar último valor de "Constucto"
new_value = "Combinación de los constructos #1 a #5"
tabla2.iloc[-1, 0] = new_value

# Check
tabla2

In [ ]:
# exportar
tabla2_path = "./Anexo_5_carpeta_1_resultados/tabla_2_search_results.csv"
tabla2.to_csv(tabla2_path, index=False, sep=";")

***
# Tabla 3: resumen de los estudios incluidos

In [ ]:
# importar studies extraction
st_ex_path = '../Anexo_4_full_text_screening/Anexo_4_carpeta_1_resultados/tabla_4.4_studies_extraction.csv'
tabla3 = pd.read_csv(st_ex_path, sep=";", index_col=False)

# limpiamos article_id (1º autor + año)
col = "article_id" # la dejamos como 1º apellido del 1º autor y año con udf importada
tabla3[col] = tabla3[col].apply(tf.extraer_apellido_y_ano).astype(str) # tb casteamos la columna
tabla3[col] = tabla3[col] + "\n" + tabla3["journal"].astype(str) # le sumamos el journal casteado

# corregimos valores mal formateados
val1 = "BBA advances"
val2 = "Journal of the American Society of Nephrology : JASN"
for x in tabla3[col]:
    if val1 in x:
        tabla3.loc[tabla3[col]==x, col] = x.replace(val1, "BBA Advances")
    elif x.endswith(val2):
        tabla3.loc[tabla3[col]==x, col] = x.replace(val2, "Journal of the American Society of Nephrology")    
    else:
        tabla3.loc[tabla3[col]==x, col] = x.title()

# check
tabla3[col].value_counts()

In [ ]:
# combinamos n case y n control en una
col = "Sample (Case/Control)" # la creamos
tabla3[col] = tabla3["sample_size_adpkd"] + " / " + tabla3["sample_size_control"]

# modificamos valores de study_design
col = "study_design" 
tabla3[col] = tabla3[col].replace({"Longitudinales": "Longitudinal", "Mixtos (clínica + in vitro)": "Mixto (clínica + in vitro)"})

# ordenar por año (ascendente)
tabla3 = tabla3.sort_values(by="year", ascending=True)

# borrar columnas innecesarias
cols_2drop = ['author', 'year', 'journal', 'title', 'doi', 'inclusion_criteria', 'biomarker_type', 'translational_apps', "sample_size_adpkd", "sample_size_control", "main_findings", "limitations"]
tabla3 = tabla3.drop(columns=cols_2drop)

# reordenar columnas: transformo a lista e imprimo
cols = list(tabla3.columns)
cols

In [ ]:
# reordeno y reasigno
cols = ['article_id', 'study_design', 'Sample (Case/Control)', 'conf_factors', 'omics_technique']
tabla3 = tabla3[cols]
cols = ["article_id","omics_technique"]
text = "procesamiento de imagen"
c = tabla3["omics_technique"].str.contains(text)
tabla3.loc[c, cols]

Ahora dejo la columna de ómicas y técnicas en formato agradable a la vista

In [ ]:
# manualmente, convierto las listas de omicas y tecnicas en una lista de dicts (para jerarquizar)
lista_dicts = [
    
    # salih 2016
    {'proteómica':['inmunoblot', 'MS'], 
     'análisis de enriquecimiento funcional':[]},
    
    # raptis 2018
    {},
    
    # rauniyar 2018
    {'proteómica':['MS'], 
     'machine learning':["clustering jerárquico"],
     'análisis de enriquecimiento funcional':[]},
    
    # evenepoel 2019
    {},
    
    # kocyigit 2019
    {'genómica':["NGS"],
     'procesamiento de imagen':[]},
    
    # raby 2021
    {'análisis de enriquecimiento funcional':[],
     'procesamiento de imagen':[],
     'proteómica':['MS', 'inmunoblot']},
    
    # dekker 2022
    {'metabolómica':['espectroscopía de RMN'],
     'procesamiento de imagen':[],
     "genómica": ["NGS"]},
    
    # kim 2022
    {'transcriptómica':['RNA-seq', 'RT-PCR']},
    
    # kocyigit 2022
    {'transcriptómica':['qRT-PCR']},
    
    # fang 2023
    {'análisis de enriquecimiento funcional':[], 'transcriptómica':['qRT-PCR'], 'análisis de predicción y construcción de redes de interacción':[]},
    
    # zhang 2024
    {'genómica':['GWAS'], 'bioestadística':["randomización mendeliana"]},
    
    # van heugten 2024
    {'proteómica':['inmunoblot', 'MS'], 'análisis de enriquecimiento funcional':[], 'transcriptómica':['snRNA-seq']}
]

# añado el dict como columna y lo transformo en columna nueva
tabla3['datos_omicas'] = lista_dicts
tabla3['bullets_omicas'] = tabla3['datos_omicas'].apply(tf.formato_bullets)

# checkeo estas 2 cols
tabla3.iloc[:,3:]

In [ ]:
# check (tabla entera)
tabla3

In [ ]:
# exportar
tabla3_path = "./Anexo_5_carpeta_1_resultados/tabla_3_articles.csv"
tabla3.to_csv(tabla3_path, index=False, sep=";")

***
# Tabla 4: biomarcadores incluidos

In [ ]:
# importar studies extraction
bm_ex_path = '../Anexo_4_full_text_screening/Anexo_4_carpeta_1_resultados/tabla_4.5_biomarkers_extraction.csv'
tabla4 = pd.read_csv(bm_ex_path, sep=";", index_col=False)

# limpiamos article_id (1º autor + año)
col = "article_id"
tabla4[col] = tabla4[col].apply(tf.extraer_apellido_y_ano)

# creamos columna de año (para odernar dataframe)
col = "year"
tabla4[col] = tabla4["article_id"].apply(tf.extraer_ano)

# ordenar por año (ascendente)
tabla4 = tabla4.sort_values(by="year", ascending=True)

# eliminar columnas innecesarias
cols_2drop = ["advantages", "year"]
tabla4 = tabla4.drop(columns=cols_2drop)

# reordenar columnas
cols = list(tabla4.columns)
cols

In [ ]:
# reordeno y reasigno
cols = ['article_id', 'biomarker_name', 'biomarker_type', 'sample_type', 'detection_technique', 'disease_stage', 'clinical_application', 'limitations', 'key_results']
tabla4 = tabla4[cols]

# normalizar valores de limitations
text_2rpl = " en estudios prospectivos más grandes"
tabla4["limitations"] = tabla4["limitations"].str.replace(text_2rpl, ".", regex=False)

# checkear para reajustar valores
col = "limitations"
vs = tabla4[col].value_counts(dropna=False)

for i in range(1,len(vs)):
    print(vs.index[i])

In [ ]:
# definimos nuevos valores
nu_vals = ["Implicaciones clínicas aún por definir.\nPendiente de validación y estandarización.", 
           "Es importante ajustar por factores de confusión.\nPendiente de validación y estandarización.", 
           "Útil solo para fases avanzadas de ADPKD.\nPendiente de validación y estandarización.", 
           "Su especificidad está en actual discusión.\nPendiente de validación y estandarización."]

# y los cambiamos con un bucle for
for i, val in zip(range(1,len(vs)), nu_vals):
    c = tabla4[col] == vs.index[i]
    tabla4.loc[c, col] = val

# y checkeo
tabla4[col].value_counts(dropna=False)

In [ ]:
# en "detection_technique", corrijo errata en "Inmunoblot"
col = "detection_technique"
tabla4[col] = tabla4[col].str.replace("Inmunoblot", "inmunoblot")

# en col "key_results", introduzco saltos de línea para hacerla más legible
col = "key_results"
tabla4[col] = tabla4[col].str.replace(". ", ".\n")

# en col "biomarker_type" sustituyo "(Vesícula Extracelular)" por "VE"
col = "biomarker_type"
tabla4[col] = tabla4[col].str.replace("(Vesícula Extracelular)", "VE")

# sustituyo np.nan por ""
tabla4 = tabla4.replace(np.nan, "")

In [ ]:
# Creamos las nuevas columnas fusionadas
# Fusión 1
tabla4['type_sample_tech'] = tabla4['biomarker_type'] + ' (' + tabla4['sample_type'] + ') / ' + tabla4['detection_technique']

# Fusión 2
tabla4['app_stage'] = tabla4['clinical_application'] + ' (' + tabla4['disease_stage'] + ')'

# check
tabla4

***
recuentos interesantes

#### biomarker_type

In [ ]:
col = "biomarker_type"
x = tabla4[col].value_counts(dropna=False)
print(sum(x.values))
x

#### sample_type

In [ ]:
col = "sample_type"
tabla4[col].value_counts(dropna=False)

#### detection_technique

In [ ]:
# eliminar columnas que acabamos de fusionar
cols_2drop = ["sample_type", "detection_technique", "clinical_application", 'disease_stage']
tabla4 = tabla4.drop(columns=cols_2drop)

# reordenar columnas
cols = list(tabla4.columns)
cols

#### biomarker_name

In [ ]:
col = "biomarker_name"
tabla4[col].value_counts(dropna=False)

# checkear col
x=str(list(tabla4[col]))
x

In [ ]:
# traducir col al inglés
eng_x = ['Complement C3', 'Villin-1', 'Periplakin', 'Envoplakin', 'Complement C9', 'HIF-1a', 'APOA1', 'APOA2', 
         'APOA4', 'DPEP1', 'EGF', 'Endocan', 'GAS6', 'Angiopoietin-2', 'TRAP5b', 'Sclerostin', 'Bone Mineral Density (BMD)', 
         'Metabolic Syndrome Criteria', 'BsAP', 'HSP90', 'Fetuina-A', 'SNX18', 'AVPR2', 'VIP36', 'Betaine', 'TERRA expression level',
         'Telomere Length', 'Osteopontin', 'myoinositol/citrate ratio', 'alanine/citrate ration', 'Phenylacetyl glycine', 'Citrate',
         'TMED5 (mRNA)', 'RUNX1T1 (mRNA)', 'PRKCE (mRNA)', 'miR-199a-3p', 'miR-107', 'FGF2 (mRNA)', 'MMP7']

# converir lista a un solo string y corregir formato
eng_x = str(eng_x).replace("[", "").replace("]", "").replace("'", "").replace("\n", "")
eng_x

In [ ]:
# reordeno y reasigno
cols = ["article_id" , 'biomarker_name', "biomarker_type", 'type_sample_tech', 'app_stage', 'key_results', 'limitations']
tabla4 = tabla4[cols]

# normalizar valores de limitations
text_2rpl = " en estudios prospectivos más grandes"
tabla4["limitations"] = tabla4["limitations"].str.replace(text_2rpl, ".", regex=False)

# checkear para reajustar valores
col = "limitations"
vs = tabla4[col].value_counts(dropna=False)

for i in range(1,len(vs)):
    print(vs.index[i])

In [ ]:
# convertir biomarker_type a lista
col = "biomarker_type"
tabla4[col] = tabla4[col].str.split()
tabla4

In [ ]:
# exportar
tabla4_path = "./Anexo_5_carpeta_1_resultados/tabla_4_biomarkers.csv"
tabla4.to_csv(tabla4_path, index=False, sep=";")